In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel, AutoModel
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch
from torch.optim import AdamW
# 
from dataset import EssayDataset
from longDataset import LongEssayDataset
from bert_regression import BertRegressionModel
from hierarchicalBert import HierarchicalBert

In [ ]:
df = pd.read_csv("dataset/aes_dataset.csv")
df.head()

# Training

In [ ]:
df_sag = df[df['dataset'] == 'sag']
df_sag.info()

In [ ]:
df_test = df_sag[:12]
df_test.info()

In [ ]:
test = df[df['max_length1'] > 1000]

In [ ]:
# load indobert tokenizer
tokenizer1 = BertTokenizer.from_pretrained("indobenchmark/indobert-lite-base-p2")

In [ ]:
# apply dataset with overlapping 128 and max length 512
test_dataset = LongEssayDataset(test, tokenizer1, 512, 128)

In [ ]:
# load dataset to dataloader for creating batch size 4
dataloader = DataLoader(test_dataset, batch_size=4, collate_fn=lambda x: list(zip(*x)))

In [ ]:
# load model, optimizer, and loss function (MSE)
model = HierarchicalBert("indobenchmark/indobert-lite-base-p2")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.MSELoss()
model.train()

In [1]:
# training process
for epoch in range(5):
    for batch, targets in dataloader:
        optimizer.zero_grad()
        # batch = batch.to(device)
        targets = torch.stack(targets).to(device)
        # forward pass
        predictions = model(batch).squeeze(1)

        print("Predictions:", predictions)
        print("Targets:", targets)
        # compute loss
        loss = criterion(predictions, targets)
        print("loss : ", loss.item())
        # backward pass
        loss.backward()
        optimizer.step()